In [ ]:
import pandas as pd
import numpy as np
import warp_norm
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
data = pd.read_csv('/home/hgh/hghData/Datasets/preprocessed_labels.csv')
image_path = data.loc[:, 'image_path']
label = data.loc[:, 'label']
results = np.loadtxt('./results.txt')

In [ ]:
img = cv2.imread(image_path[105])
print(label[105])
cleaned_string = label[105].replace('\n', '').replace(' ', '')
# 将字符串切分成单个数字的列表
numbers = cleaned_string.split('][')
numbers[0] = numbers[0].lstrip('[')
numbers[-1] = numbers[-1].rstrip(']')
# 将字符串转换为浮点数，并重新组成 NumPy 数组
numpy_array = np.array([float(num) for num in numbers]).reshape(3,1)
gaze_img = warp_norm.draw_gaze(img, numpy_array, color = (0,255,0))
gaze_img = warp_norm.draw_gaze(img, results[105])
plt.imshow(cv2.cvtColor(gaze_img, cv2.COLOR_BGR2RGB))
plt.show()

In [1]:
import os
import pandas as pd
import numpy as np
import pickle

csv_file_path = '/home/hgh/hghData/Datasets/preprocessed_labels.csv'
df = pd.read_csv(csv_file_path)
dataset_dict = {}

for index, row in df.iterrows():
    image_path = row['image_path']
    label = row['label']
    cleaned_string = label.replace('\n', '').replace(' ', '')
    # 将字符串切分成单个数字的列表
    numbers = cleaned_string.split('][')
    numbers[0] = numbers[0].lstrip('[')
    numbers[-1] = numbers[-1].rstrip(']')  
    # 组织样本信息为字典
    label_array = np.array([float(num) for num in numbers]).reshape(3,1)
    sample_info = {
        'image_path': image_path,
        'label': label_array
    }

    # 将样本信息加入数据集字典，以索引作为键
    dataset_dict[index] = sample_info

# for key, value in dataset_dict.items():
#     print(f"Sample {key}: {value}")

pickle_file_path = '/home/hgh/hghData/Datasets/dataset_dict.pkl'
with open(pickle_file_path, 'wb') as file:
    pickle.dump(dataset_dict, file)


In [36]:
import warp_norm
import numpy as np
gv = np.array([0.0685,0.3065,-0.94944])
if gv.size == 2:
    gv = warp_norm.pitchyaw_to_vector(gv)
z = np.array([0,0,-600])
theta = np.arcsin(np.linalg.norm(np.cross(gv,z))/(np.linalg.norm(gv)*np.linalg.norm(z)))
print(theta)
scale = np.linalg.norm(z)/(np.cos(theta)*np.linalg.norm(gv))
print(scale)
gp = scale * gv - z
print(gp)
gp = np.delete(gp, 2, axis=0)
t = np.array([0.215,0.215])
s = np.array([500,500])
print(gp/t+s)

0.31945605309159664
631.9514661274015
[ 4.32886754e+01  1.93693124e+02 -1.13686838e-13]
[ 701.34267642 1400.89825287]


In [39]:
import os
import pandas as pd
import numpy as np
import pickle
import warp_norm
import utils

pickle_file_path = '/home/hgh/hghData/Datasets/dataset_dict.pkl'
with open(pickle_file_path, 'rb') as file:
    loaded_dataset_dict = pickle.load(file)
ground_truth = []
for key, value in loaded_dataset_dict.items():
    # print(f"Sample {key}: {value}")
    ground_truth.append(value['label'].reshape((1,3))[0])
ground_truth = np.array(ground_truth)

ground_truth = warp_norm.vector_to_pitchyaw(-ground_truth)
print(ground_truth)

pred = []
pred_path = './results.txt'

with open(pred_path, 'r') as file:
    lines = file.readlines()

for line in lines:
    data_array = [float(x) for x in line.split()]
    pred.append(data_array)
pred = np.array(pred)
print(pred)

e = utils.angular_error(ground_truth,pred)
print('error:', np.mean(e))


[[-0.22497383 -0.2523082 ]
 [-0.23685605  0.04697659]
 [-0.35651399  0.16178618]
 ...
 [-0.13845209 -0.33410653]
 [-0.21863731  0.2093067 ]
 [-0.3591192   0.07912455]]
[[-0.14345832 -0.16445599]
 [-0.19170144 -0.01741143]
 [-0.10249488  0.09203888]
 ...
 [-0.48741466 -0.46078506]
 [-0.47450206  0.26392451]
 [-0.35260439  0.06243787]]
error: 11.809294488892363
